# Marketing Campaign Dataset - Preprocessing

## Abstract

In this notebook, **the insights from the EDA are applied** to the data. The standard data preprocessing techniques are applied in order to **end up with a final dataset** that future models can ingest. A Python script will also be developed to encapsulate all the preprocessing done in this notebook. First, the methods used to preprocess the data will be demonstrated, and then they will be put together taking into consideration the splitting of the data and avoiding data leakage.

## Table of Contents
* [Library imports and reading data](#chapter0)
* [1. Feature selection](#chapter1)
* [2. Feature engineering](#chapter2)
* [3. Outlier treatment](#chapter3)
* [4. Missing data treatment](#chapter4)
* [5. Normalization](#chapter5)
* [6. Encoding](#chapter6)
* [7. Putting everything together and splitting the data](#chapter7)

## Library imports and reading data <a class="anchor" id="chapter0"></a>

### Library imports

In [21]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# Missing data treatment
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Normalization
from sklearn.preprocessing import StandardScaler

# Split data
from sklearn.model_selection import train_test_split

# Pretty printing
from tabulate import tabulate

### Reading data

In [22]:
data_path = "../Dataset/marketing_campaign_available.csv"
data = pd.read_csv(data_path, sep=";")

## 1. Feature selection <a class="anchor" id="chapter1"></a>

### Removing ID

The variable `ID` will be removed as it does not provide any useful information to predict the target.

In [23]:
remove_list = ['ID']

### Removing variables not correlated with the target

As has been seen in the EDA notebook, the variables `Year_Birth`, `NumDealsPurchases`, `NumStorePurchases`, `NumWebVisitsMonth` and `Complain`, bear almost no relation or no relation with the target. In particular, the correlation of this variables with the target is $<0.05$, so they will be removed as they do not provide predictive power. 

In [24]:
remove_list += ['Year_Birth', 'NumDealsPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Complain']

### Removing monotonous variables

The variables `Z_CostContact` and `Z_Revenue` only consist of one unique value, this is, null variability. For this reason, they do not provide any useful information and will be thus removed from the dataset.

In [25]:
remove_list += ['Z_CostContact', 'Z_Revenue']

### Performing the feature selection

In [26]:
data.drop(remove_list, axis=1, inplace=True)

In [27]:
data.shape

(2016, 21)

Now there are only **21 variables** left.

## 2. Feature engineering <a class="anchor" id="chapter2"></a>

Given the small amount of observations for the categories Alone, YOLO and Absurd, these will be merged into a new "Others" category.

In [28]:
data['Marital_Status'].value_counts()

Marital_Status
Married     779
Together    525
Single      434
Divorced    203
Widow        69
Alone         3
YOLO          2
Absurd        1
Name: count, dtype: int64

In [29]:
data['Marital_Status'] = data['Marital_Status'].replace(['YOLO', 'Absurd', 'Alone'], 'Others')

In [30]:
data['Marital_Status'].value_counts()

Marital_Status
Married     779
Together    525
Single      434
Divorced    203
Widow        69
Others        6
Name: count, dtype: int64

Also, the variable `Dt_Customer` contains dates and they should be processed. In this case, the days that have passed from a given date to 1st January 2023 is calculated.

In [31]:
# Convert to datetime
date_objects = [datetime.strptime(date_str, "%Y-%m-%d") for date_str in data['Dt_Customer']]

# The target date
target_date = datetime(2023, 1, 1)

# Calculate the difference in days between each date and the target date
days_passed = [(target_date - date_object).days for date_object in date_objects]

# Update the column
data['Dt_Customer'] = days_passed

In [32]:
data['Dt_Customer'][0:5]

0    3394
1    3360
2    3461
3    3371
4    3335
Name: Dt_Customer, dtype: int64

## 3. Outlier treatment <a class="anchor" id="chapter3"></a>

In the EDA notebook `Income` and `Year_Birth` were observed to contain outliers. However, as `Year_Birth` has been removed, only the outliers in `Income` will be treated.

In [33]:
condition = data['Income'] > 155000
data[condition].sort_values(by='Income', ascending=False)

,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,...,MntSweetProducts,MntGoldProds,NumWebPurchases,NumCatalogPurchases,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response
638,Graduation,Together,666666.0,1,0,3500,23,9,14,18,...,1,12,3,1,0,0,0,0,0,0
216,PhD,Together,162397.0,1,1,3499,31,85,1,16,...,1,2,0,0,0,0,0,0,0,0
1757,PhD,Married,160803.0,0,0,3802,21,55,16,1622,...,3,4,0,28,0,0,0,0,0,0
1768,Master,Together,157733.0,1,0,3498,37,39,1,9,...,0,8,1,0,0,0,0,0,0,0
790,PhD,Married,157243.0,0,1,3228,98,20,2,1582,...,2,1,0,22,0,0,0,0,0,0
1354,Graduation,Together,157146.0,0,0,3534,13,1,0,1725,...,1,1,0,28,0,0,0,0,0,0
1414,PhD,Married,156924.0,0,0,3412,85,2,1,2,...,1,1,0,0,0,0,0,0,0,0


This outlier (666,666.0) is probably due to a typo, moreover, the rest of the row seems reasonable. For these reasons, the outlier in `Income` will be treated as a NA and then reconsidered in the next sections as such.

In [34]:
condition = data['Income'] == 666666
data.loc[condition, 'Income'] = np.nan

We can see how the outlier is a NaN now.

In [35]:
condition = data.index == 638
data[condition]

,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,...,MntSweetProducts,MntGoldProds,NumWebPurchases,NumCatalogPurchases,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response
638,Graduation,Together,NaN,1,0,3500,23,9,14,18,...,1,12,3,1,0,0,0,0,0,0


## 4. Missing data treatment <a class="anchor" id="chapter4"></a>

All the missing data is in `Income`. To treat the missing data, the Iterative Imputer from Sklearn will be used. This imputer, as the corresponding Sklearn page says, is a "multivariate imputer that **estimates each feature from all the others**". To perform the imputation, only numeric columns will be used. 

In [36]:
# Select numeric columns
numeric_cols = data.select_dtypes(include=np.number).columns.tolist()

# Initialize the imputer
iter_imputer = IterativeImputer(random_state=42)

# Perform the imputation
data[numeric_cols] = iter_imputer.fit_transform(data[numeric_cols])

In [37]:
missing_count = data.isnull().sum()
missing_count

Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumWebPurchases        0
NumCatalogPurchases    0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Response               0
dtype: int64

## 5. Normalization <a class="anchor" id="chapter5"></a>

The standard scaler from Sklearn will be used. This scalers transforms the data so that it has a mean of 0 and standard deviation of 1. The standarization will only be applied to numeric variables. 

In [38]:
# Create the scaler
scaler = StandardScaler()

# Fit and transform the data
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

In [39]:
data.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,...,MntSweetProducts,MntGoldProds,NumWebPurchases,NumCatalogPurchases,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response
0,Graduation,Together,0.699979,-0.833237,0.917209,-0.326999,-0.720470,0.052339,-0.554325,-0.453535,...,0.095509,1.974762,1.068635,-0.558419,-0.278386,-0.281477,-0.282502,-0.261462,-0.112056,-0.413197
1,2n Cycle,Together,-0.181618,1.016769,0.917209,-0.495252,0.183454,-0.862063,-0.655105,-0.656496,...,-0.507176,-0.645854,-1.090772,-0.895779,-0.278386,-0.281477,-0.282502,-0.261462,-0.112056,-0.413197
2,PhD,Married,-0.131262,-0.833237,0.917209,0.004558,-0.894301,0.162904,-0.655105,-0.607962,...,-0.651821,-0.549508,0.708734,-0.558419,-0.278386,-0.281477,-0.282502,-0.261462,-0.112056,-0.413197
3,PhD,Single,2.007831,-0.833237,-0.928193,-0.440818,-1.659160,2.950931,-0.126012,1.253986,...,-0.169673,-0.453162,-0.011068,0.116302,-0.278386,-0.281477,3.539802,3.824645,-0.112056,2.420153
4,Master,Together,0.617576,-0.833237,0.917209,-0.618968,-1.554861,1.316364,-0.655105,0.296539,...,-0.169673,0.124916,-0.370970,0.791023,-0.278386,-0.281477,-0.282502,-0.261462,-0.112056,-0.413197


## 6. Encoding <a class="anchor" id="chapter6"></a>

For `Education` and `Marital_Status`, encoding should be performed so the models can ingest the data. As they do not have many unique values, a standard encoding thechnique will be used, this is, one-hot encoding.

In [40]:
import pandas as pd

# Assuming 'categorical_column' is the column you want to encode
data = pd.get_dummies(data, columns=['Education', 'Marital_Status'], prefix=['Edu', 'MS'])

In [41]:
data.head()

,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Edu_Basic,Edu_Graduation,Edu_Master,Edu_PhD,MS_Divorced,MS_Married,MS_Others,MS_Single,MS_Together,MS_Widow
0,0.699979,-0.833237,0.917209,-0.326999,-0.720470,0.052339,-0.554325,-0.453535,0.071503,0.095509,...,False,True,False,False,False,False,False,False,True,False
1,-0.181618,1.016769,0.917209,-0.495252,0.183454,-0.862063,-0.655105,-0.656496,-0.574990,-0.507176,...,False,False,False,False,False,False,False,False,True,False
2,-0.131262,-0.833237,0.917209,0.004558,-0.894301,0.162904,-0.655105,-0.607962,-0.501105,-0.651821,...,False,False,False,True,False,True,False,False,False,False
3,2.007831,-0.833237,-0.928193,-0.440818,-1.659160,2.950931,-0.126012,1.253986,1.272132,-0.169673,...,False,False,False,True,False,False,False,True,False,False
4,0.617576,-0.833237,0.917209,-0.618968,-1.554861,1.316364,-0.655105,0.296539,-0.445692,-0.169673,...,False,False,True,False,False,False,False,False,True,False


## 7. Putting everything together and splitting the data <a class="anchor" id="chapter7"></a>

This section has nothing to do with the previous sections. The data will be read again, and the methods explained previously will be adapted to take into cosideration data leakage and other best practices to ensure the data is correctly preprocessed.

### Encapulating the methods into functions

In [42]:
def read_data(path):
    data = pd.read_csv(path, sep=";")
    return data

def feature_selection(data):
    # Variables that will be deleted from the dataset
    remove_list = ['ID', 'Year_Birth', 'NumDealsPurchases', 'NumStorePurchases',
                   'NumWebVisitsMonth', 'Complain', 'Z_CostContact', 'Z_Revenue']
    
    # Deleting the variables from the dataset
    return data.drop(remove_list, axis=1)

def feature_engineering(data):
    # Marital_Status =======================================
    # ======================================================
    
    # Merging the least frequent categories in Marital_Status
    data['Marital_Status'] = data['Marital_Status'].replace(['YOLO', 'Absurd', 'Alone'], 'Others')

    # Dt_Customer ==========================================
    # ======================================================

    # Convert to datetime
    date_objects = [datetime.strptime(date_str, "%Y-%m-%d") for date_str in data['Dt_Customer']]

    # The target date
    target_date = datetime(2023, 1, 1)

    # Calculate the difference in days between each date and the target date
    days_passed = [(target_date - date_object).days for date_object in date_objects]

    # Update the column
    data['Dt_Customer'] = days_passed
    
    return data
    
def outlier_treatment(data):
    # Determining which are outliers
    condition = data['Income'] == 666666
    
    # Setting outlier to NaN
    data.loc[condition, 'Income'] = np.nan
    
    return data

def split_data(data):
    # Separating the data from the response
    X = data.drop(['Response'], axis=1)
    y = data['Response']
    
    # Splitting the data with a fixed seed for reproducibility
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=data['Response'])
    
    return X_train, X_test, y_train, y_test

def imputation(X_train, X_test):
    # Select numeric columns
    numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()

    # Initialize the imputer
    iter_imputer = IterativeImputer(random_state=42)

    # Fit the imputer with the training data
    iter_imputer.fit(X_train[numeric_cols])
    
    # Impute the data
    X_train[numeric_cols] = iter_imputer.transform(X_train[numeric_cols])
    X_test[numeric_cols] = iter_imputer.transform(X_test[numeric_cols])
    
    return X_train, X_test

def normalization(X_train, X_test):
    # Select numeric columns
    numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    
    # Create the scaler
    scaler = StandardScaler()

    # Fit the scaler with the training data
    scaler.fit(X_train[numeric_cols])
    
    # Normalize the data
    X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
    X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
    
    return X_train, X_test

def encoding(X_train, X_test):
    # One-hot encoding for categorical variables
    X_train = pd.get_dummies(X_train, columns=['Education', 'Marital_Status'], prefix=['Edu', 'MS'])
    X_test = pd.get_dummies(X_test, columns=['Education', 'Marital_Status'], prefix=['Edu', 'MS'])
    
    return X_train, X_test

### Executing everything at once

In [43]:
# Path where the data is stored
data_path = "../Dataset/marketing_campaign_available.csv"

# Preprocessing pipeline
data = read_data(data_path)
data = feature_selection(data)
data = feature_engineering(data)
data = outlier_treatment(data)
X_train, X_test, y_train, y_test = split_data(data)
X_train, X_test = imputation(X_train, X_test)
X_train, X_test = normalization(X_train, X_test)
X_train, X_test = encoding(X_train, X_test)

### Showing the final preprocessed dataset

In [44]:
# Final preprocessed dataset information
dataset_info = [
    ["Train features shape", X_train.shape],
    ["Test features shape", X_test.shape],
    ["Train target shape", y_train.shape],
    ["Test target shape", y_test.shape]
]
print(tabulate(dataset_info, headers=["Description", "Shape"], tablefmt="fancy_grid"))

╒══════════════════════╤════════════╕
│ Description          │ Shape      │
╞══════════════════════╪════════════╡
│ Train features shape │ (1612, 29) │
├──────────────────────┼────────────┤
│ Test features shape  │ (404, 29)  │
├──────────────────────┼────────────┤
│ Train target shape   │ (1612,)    │
├──────────────────────┼────────────┤
│ Test target shape    │ (404,)     │
╘══════════════════════╧════════════╛


In [45]:
X_train.head()

,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Edu_Basic,Edu_Graduation,Edu_Master,Edu_PhD,MS_Divorced,MS_Married,MS_Others,MS_Single,MS_Together,MS_Widow
221,0.541626,-0.827875,-0.903672,0.760528,-0.712092,0.183324,0.226957,0.625248,1.034101,2.117899,...,False,False,False,False,False,False,False,False,True,False
848,-0.288282,-0.827875,0.930987,-0.183525,-1.335683,-0.457824,-0.626203,-0.585000,-0.660940,-0.634385,...,False,False,False,True,False,True,False,False,False,False
866,1.417851,-0.827875,-0.903672,0.755611,1.089393,2.007901,3.037366,2.899309,1.853681,0.346864,...,False,False,True,False,False,True,False,False,False,False
1372,-0.830928,1.023074,0.930987,1.670163,0.015431,-0.496773,-0.651296,-0.658218,-0.698194,-0.658318,...,False,False,False,True,False,False,False,False,True,False
982,-1.430518,1.023074,-0.903672,1.266973,-1.508903,-0.880263,-0.350180,-0.705594,-0.642313,-0.275392,...,True,False,False,False,False,True,False,False,False,False


In [46]:
X_test.head()

,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Edu_Basic,Edu_Graduation,Edu_Master,Edu_PhD,MS_Divorced,MS_Married,MS_Others,MS_Single,MS_Together,MS_Widow
1450,-1.248471,2.874023,-0.903672,-1.516017,-1.370327,-0.883259,-0.601110,-0.692674,-0.325657,-0.658318,...,False,True,False,False,False,True,False,False,False,False
191,0.526232,1.023074,0.930987,0.037737,-0.642804,-0.469808,-0.375273,-0.236139,-0.418792,-0.323258,...,False,False,False,True,False,False,False,False,True,False
1256,-2.101537,1.023074,0.930987,-1.191499,0.673666,-0.808359,-0.651296,-0.701287,-0.679567,-0.634385,...,False,False,False,True,False,False,False,False,True,False
1276,0.075631,1.023074,0.930987,1.675080,0.396514,0.153364,-0.550924,-0.391188,-0.586433,-0.658318,...,False,False,False,True,False,False,False,False,True,False
1444,0.814629,-0.827875,-0.903672,0.504847,-0.642804,-0.305027,-0.651296,1.478021,-0.083509,2.452960,...,False,False,False,True,False,False,False,True,False,False
